In [1]:
# QuiSaúde

## Imports

In [2]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from datetime import date
from sqlalchemy import create_engine
from IPython.display import clear_output

## Conexão

In [3]:
con = pg.connect(host='localhost', dbname= 'quisaude', user='postgres', password= '12345')

## Componentes

### Componentes Gerais

In [4]:
btn_instituicoes = widgets.Button(description="INSTITUIÇÕES")
btn_campanhas = widgets.Button(description="CAMPANHAS")
btn_servicos = widgets.Button(description="SERVIÇOS")
btn_enderecos = widgets.Button(description="ENDEREÇOS")
btn_consultas = widgets.Button(description="CONSULTAS")
btn_menu = widgets.Button(description="Menu")

### Componentes Endereço

In [5]:
lbl_enderecos = widgets.Label(value="Endereços")
btn_insert_enderecos = widgets.Button(description="Inserir")
btn_remove_enderecos = widgets.Button(description="Remover")
btn_update_enderecos = widgets.Button(description="Atualizar")
btn_get_all_enderecos = widgets.Button(description="Ver Todos")

txt_bairro_instituicao = widgets.Text(
    value='',
    placeholder='Digite o Bairro',
    description='Bairro: ',
    disabled=False
)

txt_numero_instituicao = widgets.Text(
    value='',
    placeholder='Digite o endereço',
    description='Número: ',
    disabled=False
)

txt_rua_instituicao = widgets.Text(
    value='',
    placeholder='Digite a rua da instituição',
    description='Rua: ',
    disabled=False
)

txt_id_endereco = widgets.BoundedIntText(
    placeholder=':Use o ID para updates e deletes',
    description='Id: ',
    disabled=False,
    min=1
)

### Componentes instituição

In [6]:
lbl_instituicao = widgets.Label(value="Instituições")
btn_insert_instituicao = widgets.Button(description="Inserir")
btn_remove_instituicao = widgets.Button(description="Remover")
btn_update_instituicao = widgets.Button(description="Atualizar")
btn_get_all_instituicao = widgets.Button(description="Ver Todos")

txt_nome_instituicao = widgets.Text(
    value='',
    placeholder='Digite o nome',
    description='Nome: ',
    disabled=False
)

txt_cnpj_instituicao = widgets.Text(
    value='',
    placeholder='Digite o CNPJ',
    description='CNPJ: ',
    disabled=False
)

txt_instituicao_id_endereco = widgets.BoundedIntText(
    placeholder=':Use o ID para updates e deletes',
    description='Id endereço: ',
    disabled=False,
    min=1
)

### Componentes Serviços

In [7]:
lbl_servico = widgets.Label(value="Serviços")
btn_insert_servico = widgets.Button(description="Inserir")
btn_remove_servico = widgets.Button(description="Remover")
btn_update_servico = widgets.Button(description="Atualizar")
btn_get_all_servico = widgets.Button(description="Ver Todos")

txt_nome_servico = widgets.Text(
    value='',
    placeholder='Digite o nome',
    description='Nome: ',
    disabled=False
)

txt_descricao_servico = widgets.Textarea(
    value='',
    placeholder='Digite a Descrição',
    description='Descrição: ',
    disabled=False
)

txt_preco_servico = widgets.FloatText(
    description='Preço: ',
    disabled=False,
    min=0
)

txt_cnpj_instituicao_servico = widgets.Text(
    value='',
    placeholder='Digite o CNJP da instituição',
    description='CNPJ Instituição: ',
    disabled=False
)

txt_id_servico = widgets.BoundedIntText(
    placeholder=':Use o ID para updates e deletes',
    description='Id: ',
    disabled=False,
    min=1
)

### Componentes de Campanha

In [8]:
lbl_campanha = widgets.Label(value="Campanhas")
btn_insert_campanha = widgets.Button(description="Inserir")
btn_remove_campanha = widgets.Button(description="Remover")
btn_update_campanha = widgets.Button(description="Atualizar")
btn_get_all_campanha = widgets.Button(description="Ver Todos")

txt_titulo_campanha = widgets.Text(
    value='',
    placeholder='Digite o nome',
    description='Nome: ',
    disabled=False
)

txt_descricao_campanha = widgets.Textarea(
    value='',
    placeholder='Digite a Descrição',
    description='Descrição: ',
    disabled=False
)

txt_id_endereco_campanha = widgets.BoundedIntText(
    description='Id Endereço: ',
    disabled=False,
    min=1
)

txt_id_campanha = widgets.BoundedIntText(
    description='Id: ',
    disabled=False,
    min=1
)

txt_cnpj_instituicao_campanha = widgets.Text(
    value='',
    placeholder='Digite o CNJP da instituição',
    description='CNPJ Instituição: ',
    disabled=False
)

data_inicio_campanha = widgets.DatePicker(
    description="Data de início: ",
    disabled=False
)

data_final_campanha = widgets.DatePicker(
    description="Data de término: ",
    disabled=False
)

### Componentes de Consultas

In [9]:
dropdown_query = widgets.Dropdown(
    options=[
        ("Quais as instituições de saúde da cidade?", 0),
        ("Quais os serviços prestados por cada instituição?", 1),
        ("Quais as campanhas vigentes no momento atual?", 2),
        ("Quais as instituições que prestam um determinado serviço?", 3),
        ("Quais médicos prestam atendimento em uma instituição?", 4),
        ("Quais os horários de atendimento de uma instituição de saúde?", 5),
        ("Quantas campanhas cada instuição tem?", 6),
        ("Qual o endereço de uma determinada instituição médica?", 7),
        ("Qual o contato de uma determinada instituição?", 8),
        ("Quais serão as próximas campanhas realizadas e suas datas?", 9)
    ],
    description="Consulta: ",
    layout={'width': 'max-content'},
)

txt_search_bar = widgets.Text(
    value='',
    placeholder='O termo para pesquisa',
    description='Pesquisa: ',
    disabled=False
)

btn_search = widgets.Button(description="Buscar")
btn_search_medicos_instituicao = widgets.Button(description="Buscar")
btn_search_horarios_instituicao = widgets.Button(description="Buscar")
btn_search_contatos_instituicao = widgets.Button(description="Buscar")
btn_search_enderecos_instituicao = widgets.Button(description="Buscar")

## CRUDs

### CRUD Endereço

In [10]:
def get_all_enderecos(b):
    select = pd.read_sql("SELECT * FROM endereco", con)
    display_enderecos_crud(b)
    display(select)

def insert_enderecos(b):
    try:
        cursor = con.cursor()
        cursor.execute("INSERT INTO endereco(rua, bairro, numero) VALUES (%s, %s, %s)",
                       (txt_rua_instituicao.value, txt_bairro_instituicao.value, txt_numero_instituicao.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_enderecos_crud(b)
        display("Não foi possível realizar a inserção. Verifique as restrições")
    finally:
        cursor.close()

def delete_enderecos(b):
    try:
        cursor =  con.cursor()
        cursor.execute("DELETE FROM endereco WHERE id = %s",
                       [txt_id_endereco.value])
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_enderecos_crud()
        display("Não foi possível realizar a deleção. Verifique as restrições")
    finally:
        cursor.close()

def update_enderecos(b):
    try:
        cursor =  con.cursor()
        cursor.execute("UPDATE endereco SET rua = %s, bairro = %s, numero = %s WHERE id = %s",
                       (txt_rua_instituicao.value, txt_bairro_instituicao.value, txt_numero_instituicao.value, txt_id_endereco.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_enderecos_crud(b)
        display("Não foi possível realizar a atualização. Verifique as restrições")
    finally:
        cursor.close()

### CRUD Instituição

In [11]:
def get_all_instituicao(b):
    select = pd.read_sql("SELECT * FROM instituicao_saude", con)
    display_instituicoes_crud(b)
    display(select)

def insert_instituicao(b):
    try:
        cursor = con.cursor()
        cursor.execute("INSERT INTO instituicao_saude(cnpj, nome, id_endereco) VALUES (%s, %s, %s)",
                       (txt_cnpj_instituicao.value, txt_nome_instituicao.value, txt_instituicao_id_endereco.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_instituicoes_crud(b)
        display("Não foi possível realizar a inserção. Verifique as restrições")
    finally:
        cursor.close()

def delete_instituicao(b):
    try:
        cursor =  con.cursor()
        cursor.execute("DELETE FROM instituicao_saude WHERE cnpj = %s",
                       [txt_cnpj_instituicao.value])
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_instituicoes_crud()
        display("Não foi possível realizar a deleção. Verifique as restrições")
    finally:
        cursor.close()

def update_instituicao(b):
    try:
        cursor =  con.cursor()
        cursor.execute("UPDATE instituicao_saude SET nome = %s, id_endereco = %s WHERE cnpj = %s",
                       (txt_nome_instituicao.value, txt_instituicao_id_endereco.value, txt_cnpj_instituicao.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_instituicoes_crud(b)
        display("Não foi possível realizar a atualização. Verifique as restrições")
    finally:
        cursor.close()

### CRUD Serviço

In [12]:
def get_all_servico(b):
    select = pd.read_sql("SELECT * FROM servico", con)
    display_servicos_crud(b)
    display(select)

def insert_servico(b):
    try:
        cursor = con.cursor()
        cursor.execute("INSERT INTO servico(nome, descricao, preco, cnpj_instituicao_saude) VALUES (%s, %s, %s, %s)",
                       (txt_nome_servico.value, txt_descricao_servico.value, txt_preco_servico.value, txt_cnpj_instituicao_servico.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_servicos_crud(b)
        display("Não foi possível realizar a inserção. Verifique as restrições")
    finally:
        cursor.close()

def delete_servico(b):
    try:
        cursor =  con.cursor()
        cursor.execute("DELETE FROM servico WHERE id = %s",
                       [txt_id_servico.value])
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_servicos_crud(b)
        display("Não foi possível realizar a deleção. Verifique as restrições")
    finally:
        cursor.close()

def update_servico(b):
    try:
        cursor =  con.cursor()
        cursor.execute("UPDATE servico SET nome = %s, descricao = %s, preco = %s WHERE id = %s",
                       (txt_nome_servico.value, txt_descricao_servico.value, txt_preco_servico.value, txt_id_servico.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_servicos_crud(b)
        display("Não foi possível realizar a atualização. Verifique as restrições")
    finally:
        cursor.close()

### CRUD Campanha

In [13]:
def get_all_campanha(b):
    select = pd.read_sql("SELECT * FROM campanha", con)
    display_campanha_crud(b)
    display(select)

def insert_campanha(b):
    try:
        cursor = con.cursor()
        cursor.execute("INSERT INTO campanha(titulo, descricao, id_endereco, cnpj_instituicao, data_inicio, data_final) VALUES (%s, %s, %s, %s, %s, %s)",
                       (txt_titulo_campanha.value, txt_descricao_campanha.value, txt_id_endereco_campanha.value, txt_cnpj_instituicao_campanha.value, data_inicio_campanha.value, data_final_campanha.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_campanha_crud(b)
        display("Não foi possível realizar a inserção. Verifique as restrições")
    finally:
        cursor.close()

def delete_campanha(b):
    try:
        cursor =  con.cursor()
        cursor.execute("DELETE FROM campanha WHERE id = %s",
                       [txt_id_campanha.value])
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_campanha_crud(b)
        display("Não foi possível realizar a deleção. Verifique as restrições")
    finally:
        cursor.close()

def update_campanha(b):
    try:
        cursor =  con.cursor()
        cursor.execute("UPDATE campanha SET titulo = %s, descricao = %s, data_inicio = %s, data_final = %s WHERE id = %s",
                       (txt_titulo_campanha.value, txt_descricao_campanha.value, data_inicio_campanha.value, data_final_campanha.value, txt_id_campanha.value))
        con.commit()
    except:
        cursor.execute("ROLLBACK")
        clear_all_output()
        display_campanha_crud(b)
        display("Não foi possível realizar a atualização. Verifique as restrições")
    finally:
        cursor.close()

## Consultas

In [14]:
def query_instituicoes_cidade(b):
    select = pd.read_sql("SELECT * FROM nomes_instituices", con)
    display_consultas(b)
    display(select)

def query_servicos_por_instituicao(b):
    select = pd.read_sql("SELECT * FROM servicos_e_instituicoes", con)
    display_consultas(b)
    display(select)

def query_campanhas_data(b):
    query = "SELECT * FROM campanhas_vigentes"
    select = pd.read_sql(query, con)
    display_consultas(b)
    display(select)

def query_instituicoes_servico(b):
    query = "SELECT * FROM servicos_e_instituicoes WHERE UPPER(nome_servico) LIKE UPPER('%" + txt_search_bar.value + "%')"

    select = pd.read_sql(query, con)
    display_consultas(b)
    display_buscar_servico(b)
    display(select)
    
def query_medicos_instituicao(b):
    query = "SELECT * FROM medico_instituicao WHERE UPPER(instituicao) LIKE UPPER('%" + txt_search_bar.value + "%')"
    
    select = pd.read_sql(query, con)
    display_consultas(b)
    display_medicos_instituicao(b)
    display(select)
    
def query_horarios_instituicao(b):
    query = "SELECT * FROM horario_instituicao WHERE UPPER(nome) LIKE UPPER('%" + txt_search_bar.value + "%')"
    select = pd.read_sql(query, con)
    display_consultas(b)
    display_horarios_instituicao(b)
    display(select)

def query_qnt_campanhas(b):
    query = "SELECT * FROM qnt_campanhas"
    select = pd.read_sql(query, con)
    display_consultas(b)
    display(select)

def query_enderecos_instituicao(b):
    query = "SELECT * FROM endereco_instituicao WHERE UPPER(nome) LIKE UPPER('%" + txt_search_bar.value + "%')"
    select = pd.read_sql(query, con)
    display_consultas(b)
    display_enderecos_instituicao(b)
    display(select)

def query_contatos_instituicao(b):
    query = "SELECT * FROM contato_instituicao WHERE UPPER(instituicao) LIKE UPPER('%" + txt_search_bar.value + "%')"
    select = pd.read_sql(query, con)
    display_consultas(b)
    display_contatos_instituicao(b)
    display(select)

def query_proximas_campanhas(b):
    query = "SELECT * FROM proximas_campanhas"
    select = pd.read_sql(query, con)
    display_consultas(b)
    display(select)

## Displays

### Displays Gerais

In [15]:
def clear_all_output():
    for i in range(10):
        clear_output(wait=True)

def display_menu(b):
    clear_all_output()
    display(btn_instituicoes, btn_campanhas, btn_servicos, btn_enderecos, btn_consultas)

### Display Instituições

In [16]:
def display_instituicoes_crud(b):
    clear_all_output()
    display(lbl_instituicao)
    display(txt_nome_instituicao, txt_cnpj_instituicao, txt_instituicao_id_endereco)
    display(btn_insert_instituicao, btn_remove_instituicao, btn_update_instituicao, btn_get_all_instituicao, btn_menu)

### Display Endereços

In [17]:
def display_enderecos_crud(b):
    clear_all_output()
    display(lbl_enderecos)
    display(txt_bairro_instituicao, txt_id_endereco, txt_rua_instituicao, txt_numero_instituicao)
    display(btn_insert_enderecos, btn_remove_enderecos, btn_update_enderecos, btn_get_all_enderecos, btn_menu)

### Display Serviço

In [18]:
def display_servicos_crud(b):
    clear_all_output()
    display(lbl_servico)
    display(txt_nome_servico, txt_id_servico, txt_preco_servico, txt_descricao_servico, txt_cnpj_instituicao_servico)
    display(btn_insert_servico, btn_remove_servico, btn_update_servico, btn_get_all_servico, btn_menu)

### Display Campanha

In [19]:
def display_campanha_crud(b):
    clear_all_output()
    display(lbl_campanha)
    display(txt_titulo_campanha, txt_descricao_campanha, txt_id_campanha, txt_id_endereco_campanha, txt_cnpj_instituicao_campanha, data_inicio_campanha,data_final_campanha)
    display(btn_insert_campanha, btn_remove_campanha, btn_update_campanha, btn_get_all_campanha, btn_menu)

### Display Consultas

In [20]:
def display_consultas(b):
    clear_all_output()
    display(btn_menu, dropdown_query)

def mudar_consulta(b):
    value_consulta = dropdown_query.value
    if value_consulta == 0:
        query_instituicoes_cidade(b)
    elif value_consulta == 1:
        query_servicos_por_instituicao(b)
    elif value_consulta == 2:
        query_campanhas_data(b)
    elif value_consulta == 3:
        display_buscar_servico(b)
    elif value_consulta == 4:
        display_medicos_instituicao(b)
    elif value_consulta == 5: 
        display_horarios_instituicao(b)
    elif value_consulta == 6:
        query_qnt_campanhas(b)
    elif value_consulta == 7:
        display_enderecos_instituicao(b)
    elif value_consulta == 8:
        display_contatos_instituicao(b)
    elif value_consulta == 9:
        query_proximas_campanhas(b)

def display_buscar_servico(b):
    clear_all_output()
    display_consultas(b)
    display(txt_search_bar, btn_search)
    
def display_medicos_instituicao(b):
    clear_all_output()
    display_consultas(b)
    display(txt_search_bar, btn_search_medicos_instituicao)
    
def display_horarios_instituicao(b):
    clear_all_output()
    display_consultas(b)
    display(txt_search_bar, btn_search_horarios_instituicao)

def display_enderecos_instituicao(b):
    clear_all_output()
    display_consultas(b)
    display(txt_search_bar, btn_search_enderecos_instituicao)
    
def display_contatos_instituicao(b):
    clear_all_output()
    display_consultas(b)
    display(txt_search_bar, btn_search_contatos_instituicao)


## Eventos

### Eventos gerais

In [21]:
btn_menu.on_click(display_menu)
btn_instituicoes.on_click(display_instituicoes_crud)
btn_enderecos.on_click(display_enderecos_crud)
btn_servicos.on_click(display_servicos_crud)
btn_campanhas.on_click(display_campanha_crud)
btn_consultas.on_click(display_consultas)

### Eventos de endereço

In [22]:
btn_insert_enderecos.on_click(insert_enderecos)
btn_get_all_enderecos.on_click(get_all_enderecos)
btn_remove_enderecos.on_click(delete_enderecos)
btn_update_enderecos.on_click(update_enderecos)

### Eventos de instituição

In [23]:
btn_get_all_instituicao.on_click(get_all_instituicao)
btn_insert_instituicao.on_click(insert_instituicao)
btn_remove_instituicao.on_click(delete_instituicao)
btn_update_instituicao.on_click(update_instituicao)

### Eventos de Servicos

In [24]:
btn_get_all_servico.on_click(get_all_servico)
btn_insert_servico.on_click(insert_servico)
btn_remove_servico.on_click(delete_servico)
btn_update_servico.on_click(update_servico)

### Eventos de Campanhas

In [25]:
btn_get_all_campanha.on_click(get_all_campanha)
btn_insert_campanha.on_click(insert_campanha)
btn_remove_campanha.on_click(delete_campanha)
btn_update_campanha.on_click(update_campanha)

### Eventos de consultas

In [26]:
dropdown_query.on_trait_change(mudar_consulta)
btn_search.on_click(query_instituicoes_servico)
btn_search_medicos_instituicao.on_click(query_medicos_instituicao)
btn_search_horarios_instituicao.on_click(query_horarios_instituicao)
btn_search_contatos_instituicao.on_click(query_contatos_instituicao)
btn_search_enderecos_instituicao.on_click(query_enderecos_instituicao)

C:\Users\victor.vieira\AppData\Local\Temp\ipykernel_6836\3905288489.py:1: DeprecationWarning: on_trait_change is deprecated in traitlets 4.1: use observe instead
  dropdown_query.on_trait_change(mudar_consulta)


## Main

In [27]:
clear_all_output()
display(btn_instituicoes, btn_campanhas, btn_servicos, btn_enderecos, btn_consultas)

Label(value='Campanhas')

Text(value='Teste', description='Nome: ', placeholder='Digite o nome')

Textarea(value='Teste descrição', description='Descrição: ', placeholder='Digite a Descrição')

BoundedIntText(value=1, description='Id: ', min=1)

BoundedIntText(value=1, description='Id Endereço: ', min=1)

...

Button(description='Remover', style=ButtonStyle())

Button(description='Atualizar', style=ButtonStyle())

Button(description='Ver Todos', style=ButtonStyle())

Button(description='Menu', style=ButtonStyle())

,titulo,descricao,id,id_endereco,cnpj_instituicao,data_inicio,data_final
0,Combate ao Mosquito Aedes AEgypti,Campanha de conscientização sobre o combate ao...,1,1,51.240.931/0001-06,2022-11-27,2022-12-20
1,Combate à Varióla dos macacos,Campanha de conscientizção sobre o combate a v...,2,2,26.067.033/0001-42,2022-11-29,2022-12-24
2,Combate à Raiva,Campanha de combate à raiva nos animais domést...,3,3,04.685.375/0001-93,2022-11-10,2022-12-20
3,Doação de sangue,Campanha de doação de sangue,4,4,04.685.375/0001-93,2022-11-01,2022-12-30
4,Combate à gripe,Campanha de vacinação contra a gripe,5,5,88.806.377/0001-83,2022-12-05,2022-12-31
5,Saúde da mulher,Do pré-natal à prevenção do câncer,6,6,24.786.414/0001-56,2022-08-15,2022-09-20
6,Combate a doença de chagas,Campanha de combate a doença de chagas,7,7,24.786.414/0001-56,2022-06-01,2022-06-12
7,Prevenção do Coronavírus,Campanha de conscientização sobre a proteção c...,8,8,27.771.002/0001-30,2022-01-30,2022-02-28
8,Novembro Azul,Mês da camapanha contra o câncer de próstata,9,9,35.228.777/0001-92,2022-11-27,2022-12-20
9,Dia Mundial de luta contra a AIDS,Dia voltado para a conscientização sobre a pre...,10,10,62.422.492/0001-60,2022-09-27,2022-10-11
